# SIMPLE PIPELINE

In this notebook we set-up a simple-pipeline that take metadata by elabFTW and insert them in a hdf5 file. To realize ths we will
- get the experiment metadata by elabFTW by using its API
- read the output file
- look for item type fields and use again API to decode that field
- create an empty hdf5 file
- map the metadata we are interested in, in the hdf5 fields

### Set up

In [48]:
pip install nexusformat


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [49]:
import datetime
# the python library for elabftw
import elabapi_python
import json
import csv
import os
import ast
import pprint
import h5py
from nexusformat.nexus import *

path="/Users/federica/Documents/Script/ELAB/elab/" #set up ypur path


###  API configuration

In [50]:

# replace with the URL of your instance
API_HOST_URL = 'https://nffa-di-electronic-lab.areasciencepark.it/api/v2/'
# replace with your api key
API_KEY = '5-a1dbfafa4bf091c744c1f816e65ad72c8773c79a100e256b36341fd4391c9f5d68b2b2d2f4a6e0ac02305' #put ypur key

### ElabFTW get

Now we will use the elab API to get the experiment that have id=48 and save the response  in a json file 'exp48.json'. We are not going to use the python library elabapi_python but using the package 'os' we work as we were on the shell and use the command curl.

In [51]:
os.system('curl -H "Authorization: '+API_KEY+'"'+"GET https://nffa-di-electronic-lab.areasciencepark.it/api/v2/experiments/49  -H 'accept: application/json' >"+ path+'exp49.json')  

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3120    0  3120    0     0  11284      0 --:--:-- --:--:-- --:--:-- 11304


0

The response is '0' that means everything has gone well!
Now let's look at the file.

In [52]:
with open(path+'exp49.json', "r") as jsonfile:
        dic_exp = json.load(jsonfile) 
dic_exp

{'access_key': None,
 'body': '',
 'body_html': '',
 'canread': '{"base": 30, "teams": [], "users": [], "teamgroups": []}',
 'canwrite': '{"base": 20, "teams": [], "users": [], "teamgroups": []}',
 'category': 7,
 'category_color': '615bae',
 'category_title': 'NXem_simplified',
 'comments': [],
 'content_type': 1,
 'created_at': '2024-10-10 09:50:59',
 'custom_id': None,
 'date': '2024-10-10',
 'elabid': '20241010-4a6bef7cb22f520f1e396250213b67ca7845fb1a',
 'events_start': None,
 'experiments_links': [],
 'firstname': 'Federica',
 'fullname': 'Federica Bazzocchi',
 'has_attachment': None,
 'has_comment': 0,
 'id': 49,
 'items_links': [{'itemid': 24,
   'title': 'S1',
   'elabid': '20241007-1b912060978c318c3866470cc406b7a92466ea6b',
   'category_title': 'Sample',
   'category_color': '3bc814',
   'is_bookable': 0,
   'link_state': 1}],
 'lastchangeby': 3,
 'lastname': 'Bazzocchi',
 'locked': 0,
 'locked_at': None,
 'lockedby': None,
 'metadata': '{"elabftw": {"extra_fields_groups": [{"

It is a json file that contains **all the informations** present in the experiment page (all the metadata!) Contrary to the "toy_pipeline" now the extra_fields corresponds to **2 groups**. One is the group of simple field as before, but the other refers to  the sample group and it is item type. Its value is a number and **corresponds to the item number 24 in the DB**. Therefore we will use another API to get the information of item 24 from the DB.
This time we are going to use the package elabapi-python.

In [53]:
# Configure the api client
configuration = elabapi_python.Configuration()
configuration.api_key['api_key'] = API_KEY
configuration.api_key_prefix['api_key'] = 'Authorization'
configuration.host = API_HOST_URL
configuration.debug = False
configuration.verify_ssl = False

# create an instance of the API class
api_client = elabapi_python.ApiClient(configuration)
# fix issue with Authorization header not being properly set by the generated lib
api_client.set_default_header(header_name='Authorization', header_value=API_KEY)


# Load the Items api

api_instance = elabapi_python.ItemsApi(api_client)


id = 24 # Integer | ID of the item
format =  'json'# | Get the entity in a different format like csv, pdf, eln or zip. "pdfa" means archive pdf (PDF/A), same with "zipa".
# (optional) (default to json)
json = True # Boolean | Include a full JSON export in the ZIP archive. Only applicable if format is zip(a).
 #(optional)
withTitle = True # Boolean | Include the title in the QR code. Only applicable if format is qrpng.
 #(optional)
#size = 56 # Integer | Specify the size of the QR code in pixels. Only applicable if format is qrpng.
 #(optional) (default to 250)
#changelog = True # Boolean | Toggles if the changelog should be included in PDF exports (pdf, pdfa, zip, zipa). Changelog is by default included if the export provides PDF/A, otherwise not.
 #(optional)


    # Read an item
api_response = api_instance.get_item(id)
dic_sample=eval(str(api_response))
dic_sample

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'nffa-di-electronic-lab.areasciencepark.it'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'_date': '2024-10-07',
 'access_key': None,
 'body': '',
 'body_html': '',
 'book_can_overlap': 1,
 'book_cancel_minutes': 0,
 'book_is_cancellable': 1,
 'book_max_minutes': 0,
 'book_max_slots': 0,
 'book_users_can_in_past': None,
 'canbook': '{"base": 30, "teams": [], "users": [], "teamgroups": []}',
 'canread': '{"base": 30, "teams": [], "users": [], "teamgroups": []}',
 'canwrite': '{"base": 30, "teams": [], "users": [], "teamgroups": []}',
 'category': 26,
 'category_color': '3bc814',
 'category_title': 'Sample',
 'comments': [],
 'content_type': 1,
 'created_at': '2024-10-07 18:16:25',
 'custom_id': None,
 'elabid': '20241007-1b912060978c318c3866470cc406b7a92466ea6b',
 'experiments_links': [],
 'firstname': 'Federica',
 'fullname': 'Federica Bazzocchi',
 'has_attachement': None,
 'has_comment': 0,
 'id': 24,
 'is_bookable': 0,
 'items_links': [],
 'lastchangeby': 3,
 'lastname': 'Bazzocchi',
 'locked': 0,
 'locked_at': None,
 'lockedby': None,
 'metadata': '{"elabftw": {"extra_f

As in the experiment template, we built the sample resource in such a way that all the metadata we need are in the 'value' of the 'metadata' key. Let's look at it

In [54]:
dic_sample['metadata']

'{"elabftw": {"extra_fields_groups": [{"id": 1, "name": "Fields"}, {"id": 2, "name": "Identifier"}, {"id": 3, "name": "Parent_Identifier"}]}, "extra_fields": {"Name": {"type": "text", "value": "sample_1", "group_id": 1}, "Type": {"type": "select", "value": "Experimental", "options": ["Experimental", "Simulation"], "group_id": 1, "required": true}, "Density": {"type": "number", "unit": "g/m^3", "units": ["g/m^3", "mg/cm^3"], "value": "0.01", "group_id": 1}, "Service": {"type": "text", "value": "absent", "group_id": 3}, "Service_s": {"type": "text", "value": "internal", "group_id": 2}, "Thickness": {"type": "number", "unit": "microm", "units": ["microm", "nm", "picom"], "value": "0.1", "group_id": 1}, "Atom_types": {"type": "text", "value": "C H", "group_id": 1, "required": true}, "Identifier": {"type": "text", "value": "absent", "group_id": 3}, "Description": {"type": "text", "value": "first_sample", "group_id": 1}, "Identifier_s": {"type": "text", "value": "S1", "group_id": 2}, "Is_per

It is a string...let us transform it in a dictionary. Moreover since "true" will not be recognized by python let us replace "true" with "True".

In [8]:
dic_meta_sample=eval(dic_sample['metadata'].replace('true','True') )


Let's explore 

In [55]:
for key, value in dic_meta_sample.items():
    print('key:',key)
    print(value)

key: elabftw
{'extra_fields_groups': [{'id': 1, 'name': 'Fields'}, {'id': 2, 'name': 'Identifier'}, {'id': 3, 'name': 'Parent_Identifier'}]}
key: extra_fields
{'Name': {'type': 'text', 'value': 'sample_1', 'group_id': 1}, 'Type': {'type': 'select', 'value': 'Experimental', 'options': ['Experimental', 'Simulation'], 'group_id': 1, 'required': True}, 'Density': {'type': 'number', 'unit': 'g/m^3', 'units': ['g/m^3', 'mg/cm^3'], 'value': '0.01', 'group_id': 1}, 'Service': {'type': 'text', 'value': 'absent', 'group_id': 3}, 'Service_s': {'type': 'text', 'value': 'internal', 'group_id': 2}, 'Thickness': {'type': 'number', 'unit': 'microm', 'units': ['microm', 'nm', 'picom'], 'value': '0.1', 'group_id': 1}, 'Atom_types': {'type': 'text', 'value': 'C H', 'group_id': 1, 'required': True}, 'Identifier': {'type': 'text', 'value': 'absent', 'group_id': 3}, 'Description': {'type': 'text', 'value': 'first_sample', 'group_id': 1}, 'Identifier_s': {'type': 'text', 'value': 'S1', 'group_id': 2}, 'Is_pe

Remember that we have also the metadata in the group 'fields' of the experiment template. We will define as in tpy_pipeline notebook

In [56]:
dic_meta_exp=eval(dic_exp['metadata'].replace('true','True') )
for key, value in dic_meta_exp.items():
    print('key:',key)
    print(value)

key: elabftw
{'extra_fields_groups': [{'id': 1, 'name': 'Sample'}, {'id': 2, 'name': 'Coordinate_System_Set'}, {'id': 3, 'name': 'Fields'}]}
key: extra_fields
{'Sample': {'type': 'items', 'value': 24, 'group_id': 1, 'required': True}, 'Definition': {'type': 'text', 'value': 'NXem', 'group_id': 3, 'required': True}, 'Start_Time': {'type': 'date', 'value': '2024-10-10', 'group_id': 3, 'required': True}, 'Experiment_alias': {'type': 'text', 'value': 'em_simplified_id1', 'group_id': 3, 'required': True}, 'Attribute_Definition_Version': {'type': 'text', 'value': 'v1', 'group_id': 3, 'required': True}}


Notice that in both dictionaries, corresponding to the 'elabftw' key there are the groups according to which we have organized the metadata. We have used only the group 'fields' to tell us that in that group we will put metadata that will be mapped in fields of the hdf5 file.
The 'extra_fields' key's value is a dictionary where:
- the key are the fields name
- the value are dictionary with the key type,value acquired by the field (we are interested in this!!!) and the corresponding group (in our case there is only a group, the fields one)

The 'Nexus' (incomplete) file we are going to build will take metadata both from the dic_meta_exp and from the dic_meta_sample. Why we put the 'Sample' as linked resources and we introduced a specific group? It is our internal convention to say 'In the Nexus file Sample is a group'

# Preparing  and mapping the hdf5/NeXus file

First we initialize a file, then we create the main group corresponding to the NXentry

In [57]:
f=h5py.File(path+'NXem_simplified.nxs','w')
f.attrs['default']='entry'

In [58]:
g_entry=f.create_group('entry')
g_entry.attrs['NX_class']='NX_entry'

Now we insert the fields as in the toy pipeline

In [59]:
g_entry['definition']=dic_meta_exp['extra_fields']['Definition']['value']
g_entry['start_time']=dic_meta_exp['extra_fields']['Start_Time']['value']
g_entry['experiment_alias']=dic_meta_exp['extra_fields']['Experiment_alias']['value']

But now we have to create the group for the sample and then insert its groups and fields. Its groups are the ones corresponding to the kew extra_fields_group with name different by 'fields' (our convention). 
Let's give again a look at the structure.

In [60]:
for key, value in dic_meta_sample.items():
    print('key:',key)
    print(value)

key: elabftw
{'extra_fields_groups': [{'id': 1, 'name': 'Fields'}, {'id': 2, 'name': 'Identifier'}, {'id': 3, 'name': 'Parent_Identifier'}]}
key: extra_fields
{'Name': {'type': 'text', 'value': 'sample_1', 'group_id': 1}, 'Type': {'type': 'select', 'value': 'Experimental', 'options': ['Experimental', 'Simulation'], 'group_id': 1, 'required': True}, 'Density': {'type': 'number', 'unit': 'g/m^3', 'units': ['g/m^3', 'mg/cm^3'], 'value': '0.01', 'group_id': 1}, 'Service': {'type': 'text', 'value': 'absent', 'group_id': 3}, 'Service_s': {'type': 'text', 'value': 'internal', 'group_id': 2}, 'Thickness': {'type': 'number', 'unit': 'microm', 'units': ['microm', 'nm', 'picom'], 'value': '0.1', 'group_id': 1}, 'Atom_types': {'type': 'text', 'value': 'C H', 'group_id': 1, 'required': True}, 'Identifier': {'type': 'text', 'value': 'absent', 'group_id': 3}, 'Description': {'type': 'text', 'value': 'first_sample', 'group_id': 1}, 'Identifier_s': {'type': 'text', 'value': 'S1', 'group_id': 2}, 'Is_pe

In [61]:
g_sample=g_entry.create_group('sample')
g_sample.attrs['NX_class']='NXsample'
g_identifier=g_sample.create_group('identifier')
g_identifier.attrs['NX_class']='NXidentifier'
g_parent_identifier=g_sample.create_group('parentidentifier')
g_parent_identifier.attrs['NX_class']='NXidentifier'

Let's now insert the fields: to identify them we extrapolate only the values corresponding to group id=1

In [62]:
sub_dic=dic_meta_sample['extra_fields']
for key, value in sub_dic.items():
    if value['group_id']==1:
        print(key)
        print(value)

Name
{'type': 'text', 'value': 'sample_1', 'group_id': 1}
Type
{'type': 'select', 'value': 'Experimental', 'options': ['Experimental', 'Simulation'], 'group_id': 1, 'required': True}
Density
{'type': 'number', 'unit': 'g/m^3', 'units': ['g/m^3', 'mg/cm^3'], 'value': '0.01', 'group_id': 1}
Thickness
{'type': 'number', 'unit': 'microm', 'units': ['microm', 'nm', 'picom'], 'value': '0.1', 'group_id': 1}
Atom_types
{'type': 'text', 'value': 'C H', 'group_id': 1, 'required': True}
Description
{'type': 'text', 'value': 'first_sample', 'group_id': 1}
Preparation_date
{'type': 'datetime-local', 'value': '2024-10-07T18:17', 'group_id': 1, 'required': True}


In [63]:
g_sample['name']=dic_meta_sample['extra_fields']['Name']['value']
g_sample['type']=dic_meta_sample['extra_fields']['Type']['value']
g_sample['density']=dic_meta_sample['extra_fields']['Density']['value']
g_sample['density'].attrs['units']=dic_meta_sample['extra_fields']['Density']['unit']
g_sample['thickness']=dic_meta_sample['extra_fields']['Thickness']['value']
g_sample['thickness'].attrs['units']=dic_meta_sample['extra_fields']['Thickness']['unit']
g_sample['atom_types']=dic_meta_sample['extra_fields']['Atom_types']['value']
g_sample['description']=dic_meta_sample['extra_fields']['Description']['value']
g_sample['preparation_date']=dic_meta_sample['extra_fields']['Preparation_date']['value']

Let's now insert the  group 'identifier' and its fields: to identify them we extrapolate only the values corresponding to group id=2

In [64]:
sub_dic=dic_meta_sample['extra_fields']
for key, value in sub_dic.items():
    if value['group_id']==2:
        print(key)
        print(value)

Service_s
{'type': 'text', 'value': 'internal', 'group_id': 2}
Identifier_s
{'type': 'text', 'value': 'S1', 'group_id': 2}
Is_persistent_s
{'type': 'select', 'value': 'True', 'options': ['True', 'False'], 'group_id': 2}


Notice this orrible notation (with the 's'): **Elab didn't allow to use in the same template resources with fields with the same name** and since before I created the parent_identifier one I was forced to differentiate the Identifier fields. The reason why this is not allowed  is related to the structure of the dictionary dic_meta_sample['extra_fields'] that arranges only  2 levels!!! Clearly in the hdf5/nxs structure fields belonging to different group may have the same name.

In [65]:
g_identifier['service']=dic_meta_sample['extra_fields']['Service_s']['value']
g_identifier['identifier']=dic_meta_sample['extra_fields']['Identifier_s']['value']
g_identifier['is_persistent']=dic_meta_sample['extra_fields']['Is_persistent_s']['value']

The parent idnetifier group has the same name as the idnetifier one, thus we may proceed.

In [66]:
g_parent_identifier['service']=dic_meta_sample['extra_fields']['Service']['value']
g_parent_identifier['identifier']=dic_meta_sample['extra_fields']['Identifier']['value']
g_parent_identifier['is_persistent']=dic_meta_sample['extra_fields']['Is_persistent']['value']

In [45]:
def printname(name):
    print(name)
f.visit(printname)    

entry
entry/definition
entry/experiment_alias
entry/sample
entry/sample/atom_types
entry/sample/density
entry/sample/description
entry/sample/identifier
entry/sample/identifier/identifier
entry/sample/identifier/is_persistent
entry/sample/identifier/service
entry/sample/name
entry/sample/parentidentifier
entry/sample/parentidentifier/identifier
entry/sample/parentidentifier/is_persistent
entry/sample/parentidentifier/service
entry/sample/preparation_dat
entry/sample/thickness
entry/sample/type
entry/start_time


In [46]:
f.close()

In [47]:
test=nxload(path+'NXem_simplified.nxs')
print(test.tree)

root:NXroot
  @default = 'entry'
  entry:NX_entry
    definition = 'NXem'
    experiment_alias = 'em_simplified_id1'
    sample:NXsample
      atom_types = 'C H'
      density = '0.01'
        @units = 'g/m^3'
      description = 'first_sample'
      identifier:NXidentifier
        identifier = 'S1'
        is_persistent = 'True'
        service = 'internal'
      name = 'sample_1'
      parentidentifier:NXidentifier
        identifier = 'absent'
        is_persistent = 'False'
        service = 'absent'
      preparation_dat = '2024-10-07T18:17'
      thickness = '0.1'
        @units = 'microm'
      type = 'Experimental'
    start_time = '2024-10-10'


In [ ]:
g

## Pre-Exercise 
Visualize the nxs we have created by using the online service https://myhdf5.hdfgroup.org/

## Exercise 1 
Define a function  that starting from dic_meta_sample create the correct structure group/fields.


## Exercise 2 
Starting from the response obtained by the API_Experiments insert the call to the item dynamically.
